# Subsequent Memory Effect

### Imports

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm


### Settings

In [2]:
# Paths
PROJECT_PATH = 'C:/Users/micha/projects/oscillation_vs_exponent/'


### Load data

In [91]:
# load spectral parameterization results
df = pd.read_csv(f"{PROJECT_PATH}/data/results/spectral_parameters.csv", index_col=0)
print(df.shape)
print(len(df)/8)
df.head()

(5560, 19)
695.0


,patient,chan_idx,label,pos_y,pos_x,pos_z,unique_id,material,memory,epoch,offset,knee,exponent,f_rotation,alpha_cf,alpha_pw,alpha_bw,alpha_bp,alpha_adj
0,pat02,0,A01-A02,-65.43100,61.94490,3.55955,pat02_0,words,hit,prestim,8.451107,12.903272,3.542673,10.978558,NaN,NaN,NaN,27777.899824,5664.172994
1,pat02,1,A02-A03,-70.93895,57.17765,12.15540,pat02_1,words,hit,prestim,7.595982,12.332651,3.052480,83.836257,10.839096,0.210785,2.000000,14445.161672,2948.320508
2,pat02,2,A03-A04,-75.39550,51.39440,20.94335,pat02_2,words,hit,prestim,7.056773,8.787521,2.774045,89.824561,12.850783,0.276397,2.000000,15085.969728,4269.780182
3,pat02,3,A04-A05,-78.91950,43.90980,30.13485,pat02_3,words,hit,prestim,7.595080,10.177476,2.940543,59.883041,NaN,NaN,NaN,23672.502537,2726.078761
4,pat02,4,A05-A06,-80.96735,35.21485,38.13475,pat02_4,words,hit,prestim,7.399935,9.126477,2.872949,65.871345,11.565201,0.227912,2.671217,21742.063343,3556.630628


In [92]:
# compute stimulus-evoked change in each parameter
df_pre = df.loc[df['epoch']=='prestim']
df_post = df.loc[df['epoch']=='poststim']
columns = df_pre.columns.tolist()
df_diff = df_pre.loc[:, columns[:9]]
df_diff.loc[:, columns[10:]] = df_post.loc[:, columns[10:]].values - df_pre.loc[:, columns[10:]].values
df_diff.loc[:,'f_rotation'] = df_pre.loc[:,'f_rotation']
df_diff

,patient,chan_idx,label,pos_y,pos_x,pos_z,unique_id,material,memory,offset,knee,exponent,f_rotation,alpha_cf,alpha_pw,alpha_bw,alpha_bp,alpha_adj
0,pat02,0,A01-A02,-65.43100,61.94490,3.55955,pat02_0,words,hit,-0.141194,-1.483239,-0.037759,10.978558,NaN,NaN,NaN,609.081874,-491.870279
1,pat02,1,A02-A03,-70.93895,57.17765,12.15540,pat02_1,words,hit,-0.421704,-5.132790,-0.218467,83.836257,NaN,NaN,NaN,2624.997518,-1071.256249
2,pat02,2,A03-A04,-75.39550,51.39440,20.94335,pat02_2,words,hit,-0.094881,-0.694436,-0.048501,89.824561,NaN,NaN,NaN,-4079.028779,-3872.210713
3,pat02,3,A04-A05,-78.91950,43.90980,30.13485,pat02_3,words,hit,-0.631166,-2.501609,-0.354102,59.883041,NaN,NaN,NaN,-3166.326144,2499.712748
4,pat02,4,A05-A06,-80.96735,35.21485,38.13475,pat02_4,words,hit,-0.575058,-2.853818,-0.315757,65.871345,NaN,NaN,NaN,-8575.127643,-4203.855148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4860,pat22,48,MOF_03-MOF_04,36.00000,-14.00000,3.00000,pat22_48,faces,miss,-0.095198,-1.291748,-0.058494,40.920078,NaN,NaN,NaN,-2.333237,3.492394
4861,pat22,49,MOF_04-MOF_07,40.50000,-14.00000,9.50000,pat22_49,faces,miss,-0.345755,-3.174832,-0.224530,31.937622,NaN,NaN,NaN,153.742252,43.903531
4862,pat22,50,MOF_07-MOF_08,44.50000,-14.00000,16.50000,pat22_50,faces,miss,-0.109624,-3.361556,-0.036085,17.964912,NaN,NaN,NaN,282.907547,-119.807390
4863,pat22,51,MOF_08-MOF_09,48.00000,-14.00000,23.50000,pat22_51,faces,miss,-0.346605,-7.055522,-0.206374,43.914230,NaN,NaN,NaN,2871.357135,53.149014


In [93]:
# marge with task-modulation results

# load task-modulation results
df_tm = pd.read_csv(r"C:\Users\micha\projects\oscillation_vs_exponent\data\results\ieeg_modulated_channels.csv", index_col=0)
df_tm['sig'] = np.where(df_tm['pval'] < 0.05, 1, 0) # determine significance

# join
df_merge = pd.merge(df_diff, df_tm, on=['patient','chan_idx','material','memory'])
df_merge

df_merge

,patient,chan_idx,label,pos_y,pos_x,pos_z,unique_id,material,memory,offset,...,alpha_cf,alpha_pw,alpha_bw,alpha_bp,alpha_adj,pval_uncorrected,sign,pval,sig,sig_tm
0,pat02,0,A01-A02,-65.43100,61.94490,3.55955,pat02_0,words,hit,-0.141194,...,NaN,NaN,NaN,609.081874,-491.870279,0.0360,1.0,0.0360,1,True
1,pat02,1,A02-A03,-70.93895,57.17765,12.15540,pat02_1,words,hit,-0.421704,...,NaN,NaN,NaN,2624.997518,-1071.256249,0.0280,1.0,0.0280,1,True
2,pat02,2,A03-A04,-75.39550,51.39440,20.94335,pat02_2,words,hit,-0.094881,...,NaN,NaN,NaN,-4079.028779,-3872.210713,0.0002,1.0,0.0002,1,True
3,pat02,3,A04-A05,-78.91950,43.90980,30.13485,pat02_3,words,hit,-0.631166,...,NaN,NaN,NaN,-3166.326144,2499.712748,0.0014,1.0,0.0014,1,True
4,pat02,4,A05-A06,-80.96735,35.21485,38.13475,pat02_4,words,hit,-0.575058,...,NaN,NaN,NaN,-8575.127643,-4203.855148,0.0000,1.0,0.0000,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2775,pat22,48,MOF_03-MOF_04,36.00000,-14.00000,3.00000,pat22_48,faces,miss,-0.095198,...,NaN,NaN,NaN,-2.333237,3.492394,0.1656,-1.0,0.1656,0,False
2776,pat22,49,MOF_04-MOF_07,40.50000,-14.00000,9.50000,pat22_49,faces,miss,-0.345755,...,NaN,NaN,NaN,153.742252,43.903531,0.9758,-1.0,0.9758,0,False
2777,pat22,50,MOF_07-MOF_08,44.50000,-14.00000,16.50000,pat22_50,faces,miss,-0.109624,...,NaN,NaN,NaN,282.907547,-119.807390,0.0140,-1.0,0.0140,1,True
2778,pat22,51,MOF_08-MOF_09,48.00000,-14.00000,23.50000,pat22_51,faces,miss,-0.346605,...,NaN,NaN,NaN,2871.357135,53.149014,0.0368,-1.0,0.0368,1,True


### Run 2x2 Anova

In [95]:
# find channels that are significant across material conditions
df_hit = df_merge.loc[df_merge['memory']=='hit']
sig = df_hit.groupby(['patient','chan_idx']).all().reset_index()

# add to df_merge
df_merge['task_modulated'] = np.nan
for ii in range(len(sig)):
    df_merge.loc[(df_merge['patient']==sig.loc[ii, 'patient']) & \
                (df_merge['chan_idx']==sig.loc[ii, 'chan_idx']), 'task_modulated'] \
                    = sig.loc[ii, 'sig']
    
# reduce
df_tm = df_merge.loc[df_merge['task_modulated']]

# show
print(f"N task-modulated channels: \t{int(len(df_tm)/4)}")
df_tm



N task-modulated channels: 	145


,patient,chan_idx,label,pos_y,pos_x,pos_z,unique_id,material,memory,offset,...,alpha_pw,alpha_bw,alpha_bp,alpha_adj,pval_uncorrected,sign,pval,sig,sig_tm,task_modulated
0,pat02,0,A01-A02,-65.43100,61.94490,3.55955,pat02_0,words,hit,-0.141194,...,NaN,NaN,609.081874,-491.870279,0.0360,1.0,0.0360,1,True,True
1,pat02,1,A02-A03,-70.93895,57.17765,12.15540,pat02_1,words,hit,-0.421704,...,NaN,NaN,2624.997518,-1071.256249,0.0280,1.0,0.0280,1,True,True
2,pat02,2,A03-A04,-75.39550,51.39440,20.94335,pat02_2,words,hit,-0.094881,...,NaN,NaN,-4079.028779,-3872.210713,0.0002,1.0,0.0002,1,True,True
4,pat02,4,A05-A06,-80.96735,35.21485,38.13475,pat02_4,words,hit,-0.575058,...,NaN,NaN,-8575.127643,-4203.855148,0.0000,1.0,0.0000,1,True,True
9,pat04,0,A01-A09,-58.30120,52.43075,-26.23685,pat04_0,words,hit,-1.154486,...,-0.177757,-0.195025,-92748.867239,-66233.478577,0.0000,1.0,0.0000,1,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2752,pat22,25,PI_05-PI_06,-10.50000,-40.00000,32.50000,pat22_25,faces,miss,0.051040,...,NaN,NaN,693.942032,87.677477,0.0302,-1.0,0.0302,1,True,True
2753,pat22,26,PI_06-PI_07,-11.50000,-40.00000,39.00000,pat22_26,faces,miss,0.172550,...,NaN,NaN,1891.499689,1392.457158,0.1644,-1.0,0.1644,0,False,True
2756,pat22,29,PI_09-PI_10,-11.50000,-41.00000,60.50000,pat22_29,faces,miss,0.954547,...,NaN,NaN,3590.493604,-8658.240511,0.1330,-1.0,0.1330,0,False,True
2763,pat22,36,AC_07-AC_08,23.50000,-31.00000,42.00000,pat22_36,faces,miss,-0.376376,...,NaN,NaN,979.281216,-39.880184,0.0052,-1.0,0.0052,1,True,True


In [97]:
# run 2-way anova (memory x material)

# loop over variables of interest
results = {}
for var in ['exponent', 'alpha_bp', 'alpha_adj']:
    # repeated measures anova
    results_i = sm.stats.AnovaRM(df_tm, f"{var}", 'unique_id', within=['memory', 'material']).fit()

    # aggregate
    results[var] = results_i

    # show results
    print(f"\n\n===================  {var}  ===================\n")
    print(results_i)



===================  exponent  ===================

                    Anova
                F Value Num DF  Den DF  Pr > F
----------------------------------------------
memory           5.1089 1.0000 144.0000 0.0253
material         6.9942 1.0000 144.0000 0.0091
memory:material  0.6597 1.0000 144.0000 0.4180



===================  alpha_bp  ===================

                    Anova
                F Value Num DF  Den DF  Pr > F
----------------------------------------------
memory           0.8354 1.0000 144.0000 0.3622
material         0.2014 1.0000 144.0000 0.6542
memory:material  1.5825 1.0000 144.0000 0.2104



===================  alpha_adj  ===================

                    Anova
                F Value Num DF  Den DF  Pr > F
----------------------------------------------
memory           0.9261 1.0000 144.0000 0.3375
material         0.6765 1.0000 144.0000 0.4121
memory:material  0.9941 1.0000 144.0000 0.3204



In [83]:
df_mean = df.groupby(by=['material','memory']).mean()
df_mean.loc[:, ['alpha_bp','alpha_adj','exponent']]

alpha_bp    alpha_adj  exponent
material memory                                    
faces    hit    -5066.663427 -2523.144398 -0.215413
         miss   -4987.541294 -2346.775722 -0.269649
words    hit    -6513.386412 -4244.049498 -0.165890
         miss   -5058.954016 -2999.551456 -0.191908